# Project Description

In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Toggle Code"></form>''')

In [ ]:
x == 1

## STEP 1: Environment Preparation (Top)

### 1.1 Import required libraries (Top)

In [3]:



print('>> Required libraries imported')


>> Required libraries imported


### 1.2 Prepare for reproducability (Top)

In [ ]:
# Set random seed to maintain reproducability
random_state = np.random.seed(42)

# Set the plotting style
plt.style.use('seaborn-whitegrid')

sns.set(style="whitegrid", palette="Blues", color_codes=True, font='sans-serif', font_scale=1.5)

## STEP 2: Data Preparation (Top)

### 2.1 Import data (Top)

### 2.2 Summary statistics (Top)

In [ ]:
# Describe the shape of the data
print("\nNumber of rows in the raw data: {}".format(df.shape[0]))
print("Number of columns in the raw data: {}\n".format(df.shape[1]))

In [ ]:
# Describe the df; Round values to 2 decimals
print(np.round(df.describe(), 2).T)

In [ ]:
# Identify column datatypes
print(df.dtypes)

list(set(df.dtypes.tolist()))

## STEP 3: Data Pre-processing (Top)¶

## STEP 4: Data Processing (Top)

## STEP 6: Feature Extraction (Top)¶

## STEP 7: Save Processed Data (Top)¶

### 7.1 Drop unnecessary columns (Top)

### 7.2 Save processed data (Top)